In [606]:
import pandas as pd
import statistics
import import_ipynb
import functions
from functions import scrape_average_rental_prices, calculate_median_rental_prices

# Load the dataset into a DataFrame
df = pd.read_csv('Data/AB_NYC_2019.csv')

# Rename columns for better clarity
df.rename(columns={'name': 'description', 'id': 'property_id'}, inplace=True)

# Remove rows where all values are missing
df.dropna(how='all', inplace=True)

# Fill missing values for specific columns with default values
df.fillna({'reviews_per_month': 'Unknown', 'last_review': 'Unknown'}, inplace=True)

# Update 'reviews_per_month' to 0 for listings with 0 reviews
df.loc[df['number_of_reviews'] == 0, 'reviews_per_month'] = 0

# Set 'last_review' to 'Never' for listings with 0 reviews
df.loc[df['number_of_reviews'] == 0, 'last_review'] = 'Never'

# Remove any instances of "'" (apostrophe) in the 'neighbourhood' column
df['neighbourhood_group'] = df['neighbourhood_group'].str.replace("'", "", regex=False)
df['neighbourhood_group'] = df['neighbourhood_group'].str.replace(".", "", regex=False)
df['neighbourhood_group'] = df['neighbourhood_group'].str.replace(' ', '-', regex=False)
df['neighbourhood_group'] = df['neighbourhood_group'].str.lower()

df['neighbourhood'] = df['neighbourhood'].str.replace("'", "", regex=False)
df['neighbourhood'] = df['neighbourhood'].str.replace(".", "", regex=False)
df['neighbourhood'] = df['neighbourhood'].str.replace(' ', '-', regex=False)
df['neighbourhood'] = df['neighbourhood'].str.lower()

# Define the mapping for neighbourhood name changes
neighbourhood_mapping = {
    'bay-terrace,-staten-island': 'bay-terrace',
    'east-morrisania': 'morrisania', 
    'fordham': 'fordham-manor',
    'sea-gate': 'seagate',
    'stuyvesant-town': 'stuyvesant-town-cooper-village',
    'concourse-village': 'concourse',
    'gramercy': 'gramercy-park',
    'rockaway-beach': 'rockaway-park', 
    'flatbush': 'flatbush-ditmas-park',  
}

# Replace the values in the 'neighbourhood' column using the mapping
df['neighbourhood'] = df['neighbourhood'].replace(neighbourhood_mapping)

df_cleaned = df[['property_id', 'description', 'neighbourhood', 'room_type', 'price', 'minimum_nights', 'number_of_reviews', 
                 'reviews_per_month', 'availability_365', 'host_id', 'host_name', 'calculated_host_listings_count']]

In [561]:
# Create a new DataFrame containing property-related details
df_property = df[['property_id', 'description', 'neighbourhood', 
                  'room_type', 'price', 'minimum_nights',
                  'reviews_per_month', 'availability_365', 'host_id']]

# Create a separate DataFrame containing host-related details
df_host = df[['host_id', 'host_name', 'calculated_host_listings_count']]

In [563]:
# Select the relevant columns from the original dataframe: 'neighbourhood_group' and 'neighbourhood'
df_neighbourhoods = df[['neighbourhood_group', 'neighbourhood']]

# Remove duplicate rows based on the selected columns
df_neighbourhoods = df_neighbourhoods.drop_duplicates()

# Reset the index of the dataframe to have a continuous range of index values
df_neighbourhoods = df_neighbourhoods.reset_index(drop=True)

In [45]:
df_nh_prices = pd.read_csv('Data/neighbourhood_prices.csv')

In [47]:
inflation_rate = 0.02 
years = 6

# arp = apartment renting price
df_nh_prices['arp_2019'] = df_nh_prices['median_price'] / ((1 + inflation_rate) ** years)

df_nh_prices['arp_2019'] = df_nh_prices['arp_2019'].round()

In [49]:
# Deleted 'median_price' column
del df_nh_prices['median_price']

In [51]:
# Merged both datasets
df_merged = pd.merge(df_cleaned, df_nh_prices, on='neighbourhood', how='left')

In [53]:
# Drop any null values
df_merged.dropna(how='any', inplace = True)

In [716]:
# export as csv table to use in SQL
df_merged.to_csv('AirBnB_NY_arp2019.csv', index=False)